In [1]:
from utils import *
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [2]:
X,y = load_rec_talk_mix()

In [3]:
np.unique(y)

array([-1.,  1.])

In [4]:
kfold = KFold(n_splits = 10, shuffle = True,random_state = 41)

In [5]:
vectorizer = TfidfVectorizer()

In [6]:
X= vectorizer.fit_transform(X)

In [7]:
X.shape

(7232, 67710)

In [9]:
scores = []
for train_index,test_index in kfold.split(X):
#     print((train_index).shape,(test_index).shape)
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    clf = SVC(kernel = 'linear')
    clf.fit(X_train,y_train)
    scores.append(score_err(clf,X_test,y_test))

In [10]:
scores #

[0.012430939226519389,
 0.004143646408839796,
 0.006915629322268302,
 0.008298755186721962,
 0.013831258644536604,
 0.0055325034578146415,
 0.008298755186721962,
 0.0055325034578146415,
 0.008298755186721962,
 0.008298755186721962]

## 线性支持向量机最优化问题
$$min_{w,b} \frac{1}{2} ||w||^{2}\\ s.t. y_{i}(w*x_{i}+b)-1>=0,i=1,2,3,N$$

## 凸优化问题
$$min_{w} f(w) \\ s.t. g_{i}(x)<=0 \\ h_{i}(w) = 0$$
- 其中，目标函数$f(w)$和约束函数$g_{i}(w)$都是$R_{n}$上的连续可微的凸函数，约束函数$h_{i}(w)$是$R^{n}$上的仿射函数
- 仿射函数 $f(x)$为仿射函数,如果他满足$f(x)=ax+b,a\in R^{n},b\in R,x \in R^{n}$

## 学习的对偶算法
- 应用拉格朗日对偶性，通过求解对偶问题得到原始问题的解
- 对偶问题往往更加容易求解
- 自然引入核函数，进而推广到非线性分类问题


## 构建拉格朗日函数
$$L(w,b,a)=\frac{1}{2}||w||^{2}-\sum_{i=1}^N{\alpha_{i}y_{i}(wx_{i}+b)}+\sum_{i=1}^{N}{\alpha_{i}}\tag{1.1}$$
- 是根据凸优化问题构建的
- 原始问题求解的是$$min_{\alpha}max_{w,b}L(w,b,\alpha)$$ 只有在满足<b>约束条件</b>求最大值的时候才是原来的方程
- 对偶问题$$max_{\alpha}min_{w,b}L(w,b,\alpha)$$

## 线性可分求解过程
- 将$\alpha$看做是一个固定值，分别对$w$和$b$求导得到
$$\begin{eqnarray*}w =\sum_{i=1}^{N}{\alpha_{i}y_{i}x_{i}}\tag{1.2}\\ 
\sum_{i=1}^{N}{\alpha_{i}y_{i}}=0\tag{1.3} \end{eqnarray*}$$
- 将式1-2代入到1-1中得到
$$min_{w,b}L(w,b,a)=-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}{\alpha_{i}\alpha_{j}y_{i}y_{j}(x_{i}*x_{j})}+\sum_{i=1}^{N}{\alpha_{i}}$$
- 对$min_{w,b}L(w,b,\alpha)$对$\alpha$的极大。即是
$$max_{\alpha}-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}{\alpha_{i}\alpha_{j}y_{i}y_{j}(x_{i}*x_{j})}+\sum_{i=1}^{N}{\alpha_{i}} \tag{1.4}\\ 
s.t. \sum_{i=1}^{N}{\alpha_{i}y_{i}}=0 \\
\alpha_{i}>=0
$$
- 式1.4的对偶问题为求最小值，然后使用SMO算法就可以求解$\alpha$
- 求出最优化的$\alpha^{*}$之后，代入1.3求出$w$，找到一个$\alpha_{j}>0$, 
$b^{*}=y_{j}-\sum_{i=1}^{N}{\alpha_{i}^{*}y_{i}(x_{i}x_{j})}$
- 最后的求解函数为
$$f(x)=sign(\sum_{i=1}^{N}\alpha_{i}y_{i}K(x,x_{i})+b^{*})$$

## 软间隔最大化
- 线性不可分意味着某些样本点$(x_{i},y_{i})$不能满足函数间隔大于等于1的约束条件。
- 其原始问题为：
$$min_{w,b,\xi}\frac{1}{2}||w||^{2}+C\sum_{i=1}^{N}\xi_{i}\\
s.t. y_{i}(wx_{i}+b)>=1-\xi_{i}, i =1,2,...,N \\
\xi_{i}>=0, i=1,2,...,N
$$
- 软间隔的支持向量$x_{i}$或者在间隔边界上，或者在间隔边界与分离超平面之间，或者在分离超平面误分一侧。若$\alpha_{i}<C,则\xi_{i}=0$,支持向量$x_{i}$恰好落在间隔边界上；若$\alpha_{i}=C,0<\xi_{i}<1$,则分类正确，$x_{i}$在间隔边界与分离超平面之间；若$\alpha_{i}=C,\xi_{i}=1$,则$x_{i}$在分离超平面上；若$\alpha_{i}=C,\xi_{i}>1$,则$x_{i}$位于分离超平面误分类一侧。

## 核函数
- 设$\chi$是输入空间，又称$H$为特征空间，如果存在一个从$\chi$到$H$的映射
$F(x):\chi \to H$使得对所有$x,z\in \chi$，函数$K(x,z)$满足条件$K(x,z)=F(x)F(z)$,则称$K(x,z)$为核函数，$F(x)$为映射函数


## 正定核函数的判断？

## 序列最小最优化算法SMO算法
- 是用来求解1.4中的算法
- 其是一种启发式算法，其基本思路是:如果两个变量的解都满足此优化问题的KTT条件，那么这个最优化问题的解就得到了。
- 求解目标
$$min_{\alpha} \frac{1}{2}\sum_{i=1}^{N}\sum_{j=1}^{N}{\alpha_{i}\alpha_{j}y_{i}y_{j}(x_{i}*x_{j})}-\sum_{i=1}^{N}{\alpha_{i}} \tag{1.5}\\ 
s.t. \sum_{i=1}^{N}{\alpha_{i}y_{i}}=0 \\
0=<\alpha_{i}<=C
$$
- 当训练样本全部满足KKT条件，训练结束

In [35]:
## ICDI SVM
class SVM(object):
    def __init__(self,C = 1.0, eps = 1e-3, num_iterator = 2000):
        self.C = C # 惩罚系数
        self.eps = eps # 精度,用于判断准确度
        self.b = 0
        self.num_iterator = num_iterator
    def kernel(self, x_i, x_j): # 默认使用的是线性核函数，即 F(x,y) = f(x)*f(y)
        return np.dot(self.train[x_i],self.train[x_j].transpose())
    
    # 求得的是预测的值
    def g(self,x_j):
        sum = 0
        for i,x_i in enumerate(self.train):
            if self.cmp(self.alpha[i]) == 0: # 为零
                continue
            sum += self.alpha[i] * self.target[i] * self.kernel(i,x_j);
        return sum + self.b
    
    # 求得是预测值和真实值之间的差值
    def E_fun(self, x_i):
        return self.g(x_i) - self.target[x_i]
    
    def cmp(self, x):
        if abs(x) < self.eps:
            return 0
        if x > 0:
            return 1
        return -1;
    
    # 判断是否满足KTT 条件 ，这个地方写的很好
    def _satisfy_KKT(self, i):
        ygx = self.target[i] * self.g(i)
        if abs(self.alpha[i])<self.eps:
            return ygx > 1 or ygx == 1
        elif abs(self.alpha[i]-self.C)<self.eps:
            return ygx < 1 or ygx == 1
        else:
            return abs(ygx-1) < self.eps
    
    # 更新alpha值
    def update_alpha(self,x_i, x_j): # i为2，j 为1
        if self.target[x_i] != self.target[x_j]:
            L = max(0,self.alpha[x_i] - self.alpha[x_j])
            H = min(self.C,self.C + self.alpha[x_i] - self.alpha[x_j])
        if self.target[x_i] == self.target[x_j]:
            L = max(0,self.alpha[x_i] + self.alpha[x_j] - self.C)
            H = min(self.C, self.alpha[x_i] + self.alpha[x_j])
        alpha_x_i_old = self.alpha[x_i]
        alpha_x_j_old = self.alpha[x_j]
        
        eta = self.kernel(x_i,x_i) + self.kernel(x_j,x_j) - 2 * self.kernel(x_i,x_j)
       
        self.alpha[x_i] = self.alpha[x_i] + self.target[x_i] * (self.E[x_j] - self.E[x_i]) / eta
        if self.alpha[x_i] > H:
            self.alpha[x_i] = H
        elif self.alpha[x_i] < L:
            self.alpha[x_i] = L
        
        self.alpha[x_j] = self.alpha[x_j] + self.target[x_i] * self.target[x_j] * (alpha_x_i_old - self.alpha[x_i]) 
        
        
        b1_new = - self.E[x_j] - self.target[x_j] * self.kernel(x_j,x_j) * (self.alpha[x_j] - alpha_x_j_old) - self.target[x_i] * self.kernel(x_i,x_j) * (self.alpha[x_i] - alpha_x_i_old) + self.b
        b2_new = - self.E[x_i] - self.target[x_j] * self.kernel(x_i,x_j) * (self.alpha[x_j] - alpha_x_j_old) - self.target[x_i] * self.kernel(x_i,x_i) * (self.alpha[x_i] - alpha_x_i_old) + self.b
       
        if 0 < self.alpha[x_j] and self.alpha[x_j] < self.C:
            self.b = b1_new
        elif 0 < self.alpha[x_i] and self.alpha[x_i] < self.C:
            self.b = b2_new
        else:
            self.b = (b1_new + b2_new ) / 2
        self.E[x_i] = self.E_fun(x_i)
        self.E[x_j] = self.E_fun(x_j)
    
    def choice_i_j(self):
        # 直接找违反KTT 条件的值
        index_list = [i for i in range(self.N)]

        j1_list_1 = list(filter(lambda i: self.alpha[i] > 0 and self.alpha[i] < self.C, index_list))
        j1_list_2 = list(set(index_list) - set(j1_list_1))

        j1_list = j1_list_1
        j1_list.extend(j1_list_2)

        for j in j1_list:
            if self._satisfy_KKT(j):
                continue

            E1 = self.E[j]
            max_ = (0, 0)

            for i in index_list:
                if i == j:
                    continue

                E2 = self.E[i]
                if abs(E1 - E2) > max_[0]:
                    max_ = (abs(E1 - E2), i)

            return j, max_[1]
        return -1,-1
    
    def fit(self,X,Y):
        self.alpha = np.zeros(X.shape[0]) # 初始化每一个参数
        self.train = X
        self.target = Y
        self.N = X.shape[0]
        self.E = np.array([self.E_fun(i) for i in range(self.N)])
#         while(True):
        for i in range(self.num_iterator):
            print('iterator: ', i)
            j,i = self.choice_i_j()
            if i == -1 and j == -1:
                break
            self.update_alpha(i,j)
    def predict(self,X):
        result = []
        for x in X:
            sum = 0
            for i,x_i in enumerate(self.train):
                if self.cmp(self.alpha[i]) == 0: # 为零
                    continue
                sum += self.alpha[i] * self.target[i] * np.dot(x.transpose(),x_i);
            result.append(self.cmp(sum + self.b))
        return np.array(result)

                

In [36]:
scores_my = []
for train_index,test_index in kfold.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    clf = SVM()
    clf.fit(X_train.toarray(),y_train)
    score = score_err(clf,X_test.toarray(),y_test)
    print(score)
    scores_my.append(score)

iterator:  0
iterator:  1
iterator:  2
iterator:  3
iterator:  4
iterator:  5
iterator:  6
iterator:  7
iterator:  8
iterator:  9
iterator:  10
iterator:  11
iterator:  12
iterator:  13
iterator:  14
iterator:  15
iterator:  16
iterator:  17
iterator:  18
iterator:  19
iterator:  20
iterator:  21
iterator:  22
iterator:  23
iterator:  24
iterator:  25
iterator:  26
iterator:  27
iterator:  28
iterator:  29
iterator:  30
iterator:  31
iterator:  32
iterator:  33
iterator:  34
iterator:  35
iterator:  36
iterator:  37
iterator:  38
iterator:  39
iterator:  40
iterator:  41
iterator:  42
iterator:  43
iterator:  44
iterator:  45
iterator:  46
iterator:  47
iterator:  48
iterator:  49
iterator:  50
iterator:  51
iterator:  52
iterator:  53
iterator:  54
iterator:  55
iterator:  56
iterator:  57
iterator:  58
iterator:  59
iterator:  60
iterator:  61
iterator:  62
iterator:  63
iterator:  64
iterator:  65
iterator:  66
iterator:  67
iterator:  68
iterator:  69
iterator:  70
iterator:  71
it

iterator:  555
iterator:  556
iterator:  557
iterator:  558
iterator:  559
iterator:  560
iterator:  561
iterator:  562
iterator:  563
iterator:  564
iterator:  565
iterator:  566
iterator:  567
iterator:  568
iterator:  569
iterator:  570
iterator:  571
iterator:  572
iterator:  573
iterator:  574
iterator:  575
iterator:  576
iterator:  577
iterator:  578
iterator:  579
iterator:  580
iterator:  581
iterator:  582
iterator:  583
iterator:  584
iterator:  585
iterator:  586
iterator:  587
iterator:  588
iterator:  589
iterator:  590
iterator:  591
iterator:  592
iterator:  593
iterator:  594
iterator:  595
iterator:  596
iterator:  597
iterator:  598
iterator:  599
iterator:  600
iterator:  601
iterator:  602
iterator:  603
iterator:  604
iterator:  605
iterator:  606
iterator:  607
iterator:  608
iterator:  609
iterator:  610
iterator:  611
iterator:  612
iterator:  613
iterator:  614
iterator:  615
iterator:  616
iterator:  617
iterator:  618
iterator:  619
iterator:  620
iterator: 

iterator:  1101
iterator:  1102
iterator:  1103
iterator:  1104
iterator:  1105
iterator:  1106
iterator:  1107
iterator:  1108
iterator:  1109
iterator:  1110
iterator:  1111
iterator:  1112
iterator:  1113
iterator:  1114
iterator:  1115
iterator:  1116
iterator:  1117
iterator:  1118
iterator:  1119
iterator:  1120
iterator:  1121
iterator:  1122
iterator:  1123
iterator:  1124
iterator:  1125
iterator:  1126
iterator:  1127
iterator:  1128
iterator:  1129
iterator:  1130
iterator:  1131
iterator:  1132
iterator:  1133
iterator:  1134
iterator:  1135
iterator:  1136
iterator:  1137
iterator:  1138
iterator:  1139
iterator:  1140
iterator:  1141
iterator:  1142
iterator:  1143
iterator:  1144
iterator:  1145
iterator:  1146
iterator:  1147
iterator:  1148
iterator:  1149
iterator:  1150
iterator:  1151
iterator:  1152
iterator:  1153
iterator:  1154
iterator:  1155
iterator:  1156
iterator:  1157
iterator:  1158
iterator:  1159
iterator:  1160
iterator:  1161
iterator:  1162
iterator

iterator:  1616
iterator:  1617
iterator:  1618
iterator:  1619
iterator:  1620
iterator:  1621
iterator:  1622
iterator:  1623
iterator:  1624
iterator:  1625
iterator:  1626
iterator:  1627
iterator:  1628
iterator:  1629
iterator:  1630
iterator:  1631
iterator:  1632
iterator:  1633
iterator:  1634
iterator:  1635
iterator:  1636
iterator:  1637
iterator:  1638
iterator:  1639
iterator:  1640
iterator:  1641
iterator:  1642
iterator:  1643
iterator:  1644
iterator:  1645
iterator:  1646
iterator:  1647
iterator:  1648
iterator:  1649
iterator:  1650
iterator:  1651
iterator:  1652
iterator:  1653
iterator:  1654
iterator:  1655
iterator:  1656
iterator:  1657
iterator:  1658
iterator:  1659
iterator:  1660
iterator:  1661
iterator:  1662
iterator:  1663
iterator:  1664
iterator:  1665
iterator:  1666
iterator:  1667
iterator:  1668
iterator:  1669
iterator:  1670
iterator:  1671
iterator:  1672
iterator:  1673
iterator:  1674
iterator:  1675
iterator:  1676
iterator:  1677
iterator

iterator:  147
iterator:  148
iterator:  149
iterator:  150
iterator:  151
iterator:  152
iterator:  153
iterator:  154
iterator:  155
iterator:  156
iterator:  157
iterator:  158
iterator:  159
iterator:  160
iterator:  161
iterator:  162
iterator:  163
iterator:  164
iterator:  165
iterator:  166
iterator:  167
iterator:  168
iterator:  169
iterator:  170
iterator:  171
iterator:  172
iterator:  173
iterator:  174
iterator:  175
iterator:  176
iterator:  177
iterator:  178
iterator:  179
iterator:  180
iterator:  181
iterator:  182
iterator:  183
iterator:  184
iterator:  185
iterator:  186
iterator:  187
iterator:  188
iterator:  189
iterator:  190
iterator:  191
iterator:  192
iterator:  193
iterator:  194
iterator:  195
iterator:  196
iterator:  197
iterator:  198
iterator:  199
iterator:  200
iterator:  201
iterator:  202
iterator:  203
iterator:  204
iterator:  205
iterator:  206
iterator:  207
iterator:  208
iterator:  209
iterator:  210
iterator:  211
iterator:  212
iterator: 

...

iterator:  1615
iterator:  1616
iterator:  1617
iterator:  1618
iterator:  1619
iterator:  1620
iterator:  1621
iterator:  1622
iterator:  1623
iterator:  1624
iterator:  1625
iterator:  1626
iterator:  1627
iterator:  1628
iterator:  1629
iterator:  1630
iterator:  1631
iterator:  1632
iterator:  1633
iterator:  1634
iterator:  1635
iterator:  1636
iterator:  1637
iterator:  1638
iterator:  1639
iterator:  1640
iterator:  1641
iterator:  1642
iterator:  1643
iterator:  1644
iterator:  1645
iterator:  1646
iterator:  1647
iterator:  1648
iterator:  1649
iterator:  1650
iterator:  1651
iterator:  1652
iterator:  1653
iterator:  1654
iterator:  1655
iterator:  1656
iterator:  1657
iterator:  1658
iterator:  1659
iterator:  1660
iterator:  1661
iterator:  1662
iterator:  1663
iterator:  1664
iterator:  1665
iterator:  1666
iterator:  1667
iterator:  1668
iterator:  1669
iterator:  1670
iterator:  1671
iterator:  1672
iterator:  1673
iterator:  1674
iterator:  1675
iterator:  1676
iterator

iterator:  144
iterator:  145
iterator:  146
iterator:  147
iterator:  148
iterator:  149
iterator:  150
iterator:  151
iterator:  152
iterator:  153
iterator:  154
iterator:  155
iterator:  156
iterator:  157
iterator:  158
iterator:  159
iterator:  160
iterator:  161
iterator:  162
iterator:  163
iterator:  164
iterator:  165
iterator:  166
iterator:  167
iterator:  168
iterator:  169
iterator:  170
iterator:  171
iterator:  172
iterator:  173
iterator:  174
iterator:  175
iterator:  176
iterator:  177
iterator:  178
iterator:  179
iterator:  180
iterator:  181
iterator:  182
iterator:  183
iterator:  184
iterator:  185
iterator:  186
iterator:  187
iterator:  188
iterator:  189
iterator:  190
iterator:  191
iterator:  192
iterator:  193
iterator:  194
iterator:  195
iterator:  196
iterator:  197
iterator:  198
iterator:  199
iterator:  200
iterator:  201
iterator:  202
iterator:  203
iterator:  204
iterator:  205
iterator:  206
iterator:  207
iterator:  208
iterator:  209
iterator: 

iterator:  692
iterator:  693
iterator:  694
iterator:  695
iterator:  696
iterator:  697
iterator:  698
iterator:  699
iterator:  700
iterator:  701
iterator:  702
iterator:  703
iterator:  704
iterator:  705
iterator:  706
iterator:  707
iterator:  708
iterator:  709
iterator:  710
iterator:  711
iterator:  712
iterator:  713
iterator:  714
iterator:  715
iterator:  716
iterator:  717
iterator:  718
iterator:  719
iterator:  720
iterator:  721
iterator:  722
iterator:  723
iterator:  724
iterator:  725
iterator:  726
iterator:  727
iterator:  728
iterator:  729
iterator:  730
iterator:  731
iterator:  732
iterator:  733
iterator:  734
iterator:  735
iterator:  736
iterator:  737
iterator:  738
iterator:  739
iterator:  740
iterator:  741
iterator:  742
iterator:  743
iterator:  744
iterator:  745
iterator:  746
iterator:  747
iterator:  748
iterator:  749
iterator:  750
iterator:  751
iterator:  752
iterator:  753
iterator:  754
iterator:  755
iterator:  756
iterator:  757
iterator: 

iterator:  1224
iterator:  1225
iterator:  1226
iterator:  1227
iterator:  1228
iterator:  1229
iterator:  1230
iterator:  1231
iterator:  1232
iterator:  1233
iterator:  1234
iterator:  1235
iterator:  1236
iterator:  1237
iterator:  1238
iterator:  1239
iterator:  1240
iterator:  1241
iterator:  1242
iterator:  1243
iterator:  1244
iterator:  1245
iterator:  1246
iterator:  1247
iterator:  1248
iterator:  1249
iterator:  1250
iterator:  1251
iterator:  1252
iterator:  1253
iterator:  1254
iterator:  1255
iterator:  1256
iterator:  1257
iterator:  1258
iterator:  1259
iterator:  1260
iterator:  1261
iterator:  1262
iterator:  1263
iterator:  1264
iterator:  1265
iterator:  1266
iterator:  1267
iterator:  1268
iterator:  1269
iterator:  1270
iterator:  1271
iterator:  1272
iterator:  1273
iterator:  1274
iterator:  1275
iterator:  1276
iterator:  1277
iterator:  1278
iterator:  1279
iterator:  1280
iterator:  1281
iterator:  1282
iterator:  1283
iterator:  1284
iterator:  1285
iterator

iterator:  1741
iterator:  1742
iterator:  1743
iterator:  1744
iterator:  1745
iterator:  1746
iterator:  1747
iterator:  1748
iterator:  1749
iterator:  1750
iterator:  1751
iterator:  1752
iterator:  1753
iterator:  1754
iterator:  1755
iterator:  1756
iterator:  1757
iterator:  1758
iterator:  1759
iterator:  1760
iterator:  1761
iterator:  1762
iterator:  1763
iterator:  1764
iterator:  1765
iterator:  1766
iterator:  1767
iterator:  1768
iterator:  1769
iterator:  1770
iterator:  1771
iterator:  1772
iterator:  1773
iterator:  1774
iterator:  1775
iterator:  1776
iterator:  1777
iterator:  1778
iterator:  1779
iterator:  1780
iterator:  1781
iterator:  1782
iterator:  1783
iterator:  1784
iterator:  1785
iterator:  1786
iterator:  1787
iterator:  1788
iterator:  1789
iterator:  1790
iterator:  1791
iterator:  1792
iterator:  1793
iterator:  1794
iterator:  1795
iterator:  1796
iterator:  1797
iterator:  1798
iterator:  1799
iterator:  1800
iterator:  1801
iterator:  1802
iterator

In [38]:
np.min(scores_my)

0.21023513139695715